In [1]:
import sys
from pathlib import Path

workind_directory = str(Path.cwd().parent)
if workind_directory not in sys.path:
    sys.path.append(str(workind_directory))

In [10]:
from pii_detection.probe_labels_utils import read_tokens_df

df_train = read_tokens_df("../data/train_shard.json")
df_val = read_tokens_df("../data/val_shard.json")



def containes_dot(word: str):
    if "." in word and not word.startswith(".") and not word.endswith("."):
        return True
    return False

def digit_streak_in_str(word:str):
    max_streak = 0
    streak = 0
    for c in word:
        if c.isdigit():
            streak += 1
        else:
            max_streak = max(max_streak, streak)
            streak = 0
    return max_streak

for df in [df_train, df_val]:
    df["contains_dot"] = df["tokens"].apply(containes_dot)
    df["contains_("] = df["tokens"].str.contains("(", regex=False)
    df["contains_-"] = df["tokens"].str.contains("-", regex=False)
    df["contains_digit"] = df["tokens"].apply(lambda x: any([c.isdigit() for c in x]))
    df["contains_@"] = df["tokens"].str.contains("@", regex=False)
    df["contains_http"] = df["tokens"].str.contains("http", regex=False)
    df["contains_www"] = df["tokens"].str.contains("www", regex=False)
    df["contains_.com"] = df["tokens"].str.contains(".com", regex=False)
    df["is_upper"] = df["tokens"].str.isupper()
    df["is_title"] = df["tokens"].str.istitle()
    df["is_digit"] = df["tokens"].str.isdigit()
    df["is_alpha"] = df["tokens"].str.isalpha()
    df["is_space"] = df["tokens"].str.isspace()
    df["is_lower"] = df["tokens"].str.islower()
    df["is_numeric"] = df["tokens"].str.isnumeric()
    df["is_alnum"] = df["tokens"].str.isalnum()
    df["is_decimal"] = df["tokens"].str.isdecimal()
    df["tokens_len"] = df["tokens"].str.len()
    df["new_line"] = df["tokens"].str.contains("\n")
    df["new_paragraph"] = (df["tokens"].str.count("\n") > 1)
    df["line_num"] = df.groupby("document")["new_line"].cumsum()
    df["paragraph_num"] = df.groupby("document")["new_paragraph"].cumsum()
    df["n_digits"] = df["tokens"].apply(lambda x: sum([c.isdigit() for c in x]))
    df["digit_streak"] = df["tokens"].apply(digit_streak_in_str)
# df[~df["tokens"].apply(f)]["labels"].value_counts()
# df["tokens"].apply(f).value_counts(normalize=True)

df_train["labels"] = df_train["labels"].apply(lambda x: "O" if "NAME" in x else x)
df_val["labels"] = df_val["labels"].apply(lambda x: "O" if "NAME" in x else x)
df_train.shape[1] == df_val.shape[1]

True

In [3]:
import pandas as pd


def mask(df: pd.DataFrame) -> pd.Series:
    return (df["is_title"] | df["contains_http"] | df["contains_www"] | df["contains_.com"] | df["contains_@"] | df["contains_("] | df["contains_-"] | df["contains_digit"] | df["contains_dot"])

,document,loc_in_doc,tokens,trailing_whitespace,labels,contains_dot,contains_(,contains_-,contains_digit,contains_@,...,is_numeric,is_alnum,is_decimal,tokens_len,new_line,new_paragraph,line_num,paragraph_num,n_digits,digit_streak
0,7,0,Design,True,O,False,False,False,False,False,...,False,True,False,6,False,False,0,0,0,0
1,7,1,Thinking,True,O,False,False,False,False,False,...,False,True,False,8,False,False,0,0,0,0
2,7,2,for,True,O,False,False,False,False,False,...,False,True,False,3,False,False,0,0,0,0
3,7,3,innovation,True,O,False,False,False,False,False,...,False,True,False,10,False,False,0,0,0,0
4,7,4,reflexion,False,O,False,False,False,False,False,...,False,True,False,9,False,False,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3494932,22687,815,process,True,O,False,False,False,False,False,...,False,True,False,7,False,False,13,13,0,0
3494933,22687,816,explained,True,O,False,False,False,False,False,...,False,True,False,9,False,False,13,13,0,0
3494934,22687,817,above,False,O,False,False,False,False,False,...,False,True,False,5,False,False,13,13,0,0
3494935,22687,818,.,False,O,False,False,False,False,False,...,False,False,False,1,False,False,13,13,0,0


In [4]:
# train random forest classifier:

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import numpy as np

train_mask = mask(df_train)
val_mask = mask(df_val)

X_train = df_train[train_mask].drop(["labels", "tokens", "document"], axis=1)
y_train = df_train[train_mask]["labels"]
X_val = df_val[val_mask].drop(["labels", "tokens", "document"], axis=1)
y_val = df_val[val_mask]["labels"]

clf = RandomForestClassifier(verbose=3, n_jobs=8)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_val)

print(classification_report(y_val, y_pred))
print(confusion_matrix(y_val, y_pred))
print(np.unique(y_pred))
print(np.unique(y_val))


[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.


building tree 1 of 100
building tree 2 of 100
building tree 3 of 100
building tree 4 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100


[Parallel(n_jobs=8)]: Done  16 tasks      | elapsed:    0.3s


building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
building tree 43 of 100
building tree 44 of 100
building tree 45 of 100
building tree 46 of 100
building tree 47 of 100
building tree 48 of 100
building tree 49 of 100
building tree 50 of 100
building tree 51 of 100
building tree 52 of 100
building tree 53 of 100
building tree 54 of 100
building tree 55 of 100
building tree 56 of 100
building tree 57 of 100
building tree 58 of 100
building tree 59 of 100
building tree 60 of 100
building tree 61 of 100
building tree 62 of 100
building tree 63 of 100
building tree 64 of 100
building tree 65 of 100
building tree 66 of 100
building tree 67 of 100
building tree 68 of 100
building tree 69 of 100
building tree 70

[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    1.6s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  16 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    0.1s finished


                precision    recall  f1-score   support

       B-EMAIL       1.00      1.00      1.00        10
      B-ID_NUM       1.00      0.83      0.91        24
B-URL_PERSONAL       0.59      0.59      0.59        17
             O       1.00      1.00      1.00     80000

      accuracy                           1.00     80051
     macro avg       0.90      0.86      0.87     80051
  weighted avg       1.00      1.00      1.00     80051

[[   10     0     0     0]
 [    0    20     0     4]
 [    0     0    10     7]
 [    0     0     7 79993]]
['B-EMAIL' 'B-ID_NUM' 'B-URL_PERSONAL' 'O']
['B-EMAIL' 'B-ID_NUM' 'B-URL_PERSONAL' 'O']


In [11]:
y_pred = clf.predict(X_train)
print(classification_report(y_train, y_pred))
print(confusion_matrix(y_train, y_pred))
print(np.unique(y_pred))
print(np.unique(y_train))

[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  16 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    0.3s finished


                  precision    recall  f1-score   support

         B-EMAIL       1.00      1.00      1.00        28
        B-ID_NUM       1.00      1.00      1.00        50
     B-PHONE_NUM       1.00      0.83      0.91         6
B-STREET_ADDRESS       1.00      1.00      1.00         2
  B-URL_PERSONAL       1.00      1.00      1.00        84
        I-ID_NUM       1.00      1.00      1.00         1
     I-PHONE_NUM       1.00      0.73      0.85        15
I-STREET_ADDRESS       1.00      0.50      0.67        12
               O       1.00      1.00      1.00    279427

        accuracy                           1.00    279625
       macro avg       1.00      0.90      0.94    279625
    weighted avg       1.00      1.00      1.00    279625

[[    28      0      0      0      0      0      0      0      0]
 [     0     50      0      0      0      0      0      0      0]
 [     0      0      5      0      0      0      0      0      1]
 [     0      0      0      2      0      0  

In [100]:
import json

with open("../data/train_shard.json", "r") as f:
    train_data = json.load(f)
train_data = sorted(train_data, key=lambda x: x["document"])

all_names = []
for essay in train_data:
    doc_id = essay["document"]
    names = []
    
    for token, label in zip(essay["tokens"], essay["labels"]):
        if "NAME_STUDENT" in label:
            if label.startswith("B"):
                names.append(token)
            elif label.startswith("I"):
                names[-1] += f" {token}"
    if names:
        all_names.append(names)

all_names


with open("../data/train_shard_renamed.json", "r") as f:
    train_data = json.load(f)
train_data = sorted(train_data, key=lambda x: x["document"])

all_names2 = []
for essay in train_data:
    doc_id = essay["document"]
    names = []
    
    for token, label in zip(essay["tokens"], essay["labels"]):
        if "NAME_STUDENT" in label:
            if label.startswith("B"):
                names.append(token)
            elif label.startswith("I"):
                names[-1] += f" {token}"
    if names:
        all_names2.append(names)

pd.DataFrame({"names": all_names, "names2": all_names2}).

,names,names2
0,"[Nathalie Sylla, Nathalie Sylla, Nathalie Sylla]","[Nina Leon, Nina Leon, Nina Leon]"
1,[Silvia Villalobos],[Ralph Fleming]
2,[Sakir Ahmad],[Toni Bartlett]
3,[Francisco Ferreira],[Stephanie Hansen]
4,[Pepa Medrano],[Roger Prince]
5,[Deiby],[Lydia]
6,[Claudia Sarria],[Kelli Campbell]
7,[Rajinder Santos],[Carl Hahn]
8,[Zia],[Ellen]
9,[Davide Carletti],[Marissa Moore]


In [125]:
with open("../data/train_shard_renamed.json", "r") as f:
    train_data = sorted(json.load(f), key=lambda x: x["document"])

train_data[0]

{'document': 7,
 'full_text': "Design Thinking for innovation reflexion-Avril 2021-Nathalie Sylla\n\nChallenge & selection\n\nThe tool I use to help all stakeholders finding their way through the complexity of a project is the  mind map.\n\nWhat exactly is a mind map? According to the definition of Buzan T. and Buzan B. (1999, Dessine-moi  l'intelligence. Paris: Les Éditions d'Organisation.), the mind map (or heuristic diagram) is a graphic  representation technique that follows the natural functioning of the mind and allows the brain's  potential to be released. Cf Annex1\n\nThis tool has many advantages:\n\n•  It is accessible to all and does not require significant material investment and can be done  quickly\n\n•  It is scalable\n\n•  It allows categorization and linking of information\n\n•  It can be applied to any type of situation: notetaking, problem solving, analysis, creation of  new ideas\n\n•  It is suitable for all people and is easy to learn\n\n•  It is fun and encourages

In [120]:
from transformers import BertTokenizer
# import bert-base-uncased

def tokenize_text(text, max_length=512):
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    tokens = tokenizer.encode(text, add_special_tokens=False, max_length=max_length)
    return tokenizer.convert_ids_to_tokens(tokens)

In [126]:
from transformers import DistilBertTokenizer, DistilBertForQuestionAnswering
import torch

def chat_with_bert(context, question, model, tokenizer):
    # Tokenize the input
    inputs = tokenizer(context, question, return_tensors="pt")

    # Get the model output
    outputs = model(**inputs)
    
    # Get the answer span
    start_scores = outputs.start_logits
    end_scores = outputs.end_logits

    # Find the tokens with the highest start and end scores
    start_index = torch.argmax(start_scores)
    end_index = torch.argmax(end_scores)

    # Get the answer span
    answer_span = inputs["input_ids"][0][start_index:end_index+1]
    answer = tokenizer.decode(answer_span, skip_special_tokens=True)

    return answer

# Load DistilBERT model and tokenizer
model_name = 'distilbert-base-cased-distilled-squad'
tokenizer = DistilBertTokenizer.from_pretrained(model_name)
model = DistilBertForQuestionAnswering.from_pretrained(model_name)

# Example conversation
context = "BERT is a powerful pre-trained language model. It can be fine-tuned for various natural language processing tasks."

# Chat with BERT
while True:
    question = input("You: ")
    if question.lower() in ["exit", "quit", "bye"]:
        print("Chat ended.")
        break
    
    answer = chat_with_bert(context, question, model, tokenizer)
    print("BERT:", answer)


tokenizer_config.json: 100%|██████████| 29.0/29.0 [00:00<00:00, 117kB/s]
vocab.txt: 100%|██████████| 213k/213k [00:00<00:00, 497kB/s]
tokenizer.json: 100%|██████████| 436k/436k [00:00<00:00, 907kB/s]
config.json: 100%|██████████| 473/473 [00:00<00:00, 2.53MB/s]
model.safetensors: 100%|██████████| 261M/261M [00:13<00:00, 18.7MB/s] 


BERT: BERT
BERT: BERT
BERT: BERT
BERT: BERT
BERT: BERT
BERT: BERT
BERT: BERT
BERT: BERT


KeyboardInterrupt: Interrupted by user

In [123]:
print(essay_modified["tokens"])

['Design', 'Thinking', 'for', 'innovation', 'reflexion', '-', 'Avril', '2021', '-', 'James', 'Moore', '\n\n', 'Challenge', '&', 'selection', '\n\n', 'The', 'tool', 'I', 'use', 'to', 'help', 'all', 'stakeholders', 'finding', 'their', 'way', 'through', 'the', 'complexity', 'of', 'a', 'project', 'is', 'the', ' ', 'mind', 'map', '.', '\n\n', 'What', 'exactly', 'is', 'a', 'mind', 'map', '?', 'According', 'to', 'the', 'definition', 'of', 'Buzan', 'T.', 'and', 'Buzan', 'B.', '(', '1999', ',', 'Dessine', '-', 'moi', ' ', "l'intelligence", '.', 'Paris', ':', 'Les', 'Éditions', "d'Organisation", '.', ')', ',', 'the', 'mind', 'map', '(', 'or', 'heuristic', 'diagram', ')', 'is', 'a', 'graphic', ' ', 'representation', 'technique', 'that', 'follows', 'the', 'natural', 'functioning', 'of', 'the', 'mind', 'and', 'allows', 'the', 'brain', "'s", ' ', 'potential', 'to', 'be', 'released', '.', 'Cf', 'Annex1', '\n\n', 'This', 'tool', 'has', 'many', 'advantages', ':', '\n\n', '•', ' ', 'It', 'is', 'accessib

In [121]:
from tqdm import tqdm



Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


['design',
 'thinking',
 'for',
 'innovation',
 'reflex',
 '##ion',
 '-',
 'av',
 '##ril',
 '2021',
 '-',
 'nat',
 '##hal',
 '##ie',
 'sy',
 '##lla',
 'challenge',
 '&',
 'selection',
 'the',
 'tool',
 'i',
 'use',
 'to',
 'help',
 'all',
 'stakeholders',
 'finding',
 'their',
 'way',
 'through',
 'the',
 'complexity',
 'of',
 'a',
 'project',
 'is',
 'the',
 'mind',
 'map',
 '.',
 'what',
 'exactly',
 'is',
 'a',
 'mind',
 'map',
 '?',
 'according',
 'to',
 'the',
 'definition',
 'of',
 'bu',
 '##zan',
 't',
 '.',
 'and',
 'bu',
 '##zan',
 'b',
 '.',
 '(',
 '1999',
 ',',
 'des',
 '##sin',
 '##e',
 '-',
 'moi',
 'l',
 "'",
 'intelligence',
 '.',
 'paris',
 ':',
 'les',
 'editions',
 'd',
 "'",
 'organisation',
 '.',
 ')',
 ',',
 'the',
 'mind',
 'map',
 '(',
 'or',
 'he',
 '##uri',
 '##stic',
 'diagram',
 ')',
 'is',
 'a',
 'graphic',
 'representation',
 'technique',
 'that',
 'follows',
 'the',
 'natural',
 'functioning',
 'of',
 'the',
 'mind',
 'and',
 'allows',
 'the',
 'brain',
 "